In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2025-12-04 11:36:34.610526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# all sizes are in cm
cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df =pd.read_csv('iris.data', names=cols)

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [5]:
df.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,class
count,150.000000,150.000000,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,Iris-setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.843333,3.054000,3.758667,1.198667,NaN
std,0.828066,0.433594,1.764420,0.763161,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN


In [6]:
df['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [7]:
# Encode the class labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
    
df['class'] = le.fit_transform(df['class'])

In [8]:
# Splitting the data into train,test, val sets
train, test, val = np.split(df.sample(frac=1, random_state=42), [int(.7*len(df)), int(.85*len(df))])

/Users/jokello/Projects/DS/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
# Get features and labels
def get_features_and_labels(dataframe, y_col, x_cols=None):
    if x_cols is not None:
        X = dataframe[x_cols].values
    else:
        X = dataframe.drop(y_col, axis=1).values
    y = dataframe[y_col].values.reshape(-1, 1)
    return X, y

In [10]:
# Scale data
def scale_data(dataframe, x_cols):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    dataframe[x_cols] = scaler.fit_transform(dataframe[x_cols])
    
    return dataframe

In [11]:
X_train, y_train = get_features_and_labels(train, 'class')
X_test, y_test = get_features_and_labels(test, 'class')
X_val, y_val = get_features_and_labels(val, 'class')
x_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
train = scale_data(train, x_cols)
test = scale_data(test, x_cols)
val = scale_data(val, x_cols)

Build the Neural Network

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [20]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.3143 - loss: 1.1381 - val_accuracy: 0.4783 - val_loss: 0.9547
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4857 - loss: 1.0038 - val_accuracy: 0.6957 - val_loss: 0.9113
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7143 - loss: 0.9285 - val_accuracy: 0.5217 - val_loss: 0.9054
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8667 - loss: 0.8741 - val_accuracy: 0.6957 - val_loss: 0.8198
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6952 - loss: 0.8220 - val_accuracy: 0.6957 - val_loss: 0.7554
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6857 - loss: 0.7705 - val_accuracy: 0.8261 - val_loss: 0.7345
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8857 - loss: 0.7137 - val_accuracy: 1.0000 - val_loss: 0.7119
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8857 - loss: 0.6550 - val_accuracy: 0.8261 - val_loss

In [21]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 1.0000 - loss: 0.0407


[0.040748659521341324, 1.0]